In [1]:
import torch

import torch.nn as nn

In [2]:
# Corpus and Vocabulary

corpus = "deep learning is fun and deep learning is powerful"

words = corpus.split()

vocab = sorted(set(words))

word_to_idx = {word: i for i , word in enumerate(vocab)}

idx_to_word = {i: word for word, i in word_to_idx.items()}

vocab_size = len(vocab)

In [3]:
# Training Sequences
training_sentences = [
    ['deep', 'learning', 'is', 'fun'],
    ['learning', 'is', 'powerful'],
    ['deep', 'learning', 'is', 'powerful'],
    ['learning', 'is', 'fun', 'and', 'powerful']
]

In [4]:
seq_length = 3

input_seqs = []

target_words = []

In [5]:
for sentence in training_sentences:

    if len(sentence) <= seq_length:

        continue

    for i in range(len(sentence) - seq_length):

        input_seq = sentence[i: i + seq_length]

        target_word = sentence[i + seq_length]

        input_tensor = torch.tensor([word_to_idx[word]for word in input_seq], dtype=torch.long)

        target_tensor = torch.tensor(word_to_idx[target_word], dtype=torch.long)

        input_seqs.append(input_tensor)

        target_words.append(target_tensor)

In [6]:
# LSTM Model

class WordLSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(WordLSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim,hidden_dim, batch_first=True)

        self.fc = nn.Linear(hidden_dim, vocab_size)

    
    def forward(self, x, hidden):

        embedded = self.embedding(x)

        out, hidden = self.lstm(embedded, hidden)

        out = self.fc(out[:,-1,:])

        return out, hidden

In [7]:
# Hyperparameters

embedding_dim = 10

hidden_dim = 16

num_epochs = 300

learning_rate = 0.01

In [8]:
model = WordLSTM(vocab_size, embedding_dim, hidden_dim)

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Training
for epoch in range(1, num_epochs + 1):
    total_loss = 0.0
    model.train()

    for input_tensor, target_tensor in zip(input_seqs, target_words):
        hidden = (torch.zeros(1, 1, hidden_dim), torch.zeros(1, 1, hidden_dim))  # LSTM has (h0, c0)
        input_tensor = input_tensor.unsqueeze(0)

        optimizer.zero_grad()
        output, hidden = model(input_tensor, hidden)
        loss = loss_fn(output, target_tensor.unsqueeze(0))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss:.4f}")
        model.eval()
        print("Sample Predictions:")
        with torch.no_grad():
            for input_tensor in input_seqs:
                input_tensor = input_tensor.unsqueeze(0)
                hidden = (torch.zeros(1, 1, hidden_dim), torch.zeros(1, 1, hidden_dim))
                output, _ = model(input_tensor, hidden)
                pred_idx = torch.argmax(output, dim=1).item()
                input_words = [idx_to_word[idx.item()] for idx in input_tensor.squeeze()]
                print(f'{input_words} → {idx_to_word[pred_idx]}')

Epoch 50/300, Loss: 1.4605
Sample Predictions:
['deep', 'learning', 'is'] → powerful
['deep', 'learning', 'is'] → powerful
['learning', 'is', 'fun'] → and
['is', 'fun', 'and'] → powerful
Epoch 100/300, Loss: 1.4287
Sample Predictions:
['deep', 'learning', 'is'] → powerful
['deep', 'learning', 'is'] → powerful
['learning', 'is', 'fun'] → and
['is', 'fun', 'and'] → powerful
Epoch 150/300, Loss: 1.4184
Sample Predictions:
['deep', 'learning', 'is'] → powerful
['deep', 'learning', 'is'] → powerful
['learning', 'is', 'fun'] → and
['is', 'fun', 'and'] → powerful
Epoch 200/300, Loss: 1.4131
Sample Predictions:
['deep', 'learning', 'is'] → powerful
['deep', 'learning', 'is'] → powerful
['learning', 'is', 'fun'] → and
['is', 'fun', 'and'] → powerful
Epoch 250/300, Loss: 1.4100
Sample Predictions:
['deep', 'learning', 'is'] → powerful
['deep', 'learning', 'is'] → powerful
['learning', 'is', 'fun'] → and
['is', 'fun', 'and'] → powerful
Epoch 300/300, Loss: 1.4077
Sample Predictions:
['deep', 'lea